## 🧮 Mathematical Foundation

### 1. User-Item Matrix

**Matrix R**: Users × Items, entries = ratings (or 1/0 for implicit feedback)

$$R = \begin{bmatrix}
r_{11} & r_{12} & \cdots & r_{1m} \\
r_{21} & ? & \cdots & r_{2m} \\
\vdots & \vdots & \ddots & \vdots \\
r_{n1} & r_{n2} & \cdots & ?
\end{bmatrix}$$

**Goal**: Predict missing entries (marked as ?)

**Sparsity**: Typically >99% missing (users rate <1% of items)

### 2. Similarity Metrics

**Cosine Similarity** (angle between vectors):

$$\text{sim}(u, v) = \frac{\mathbf{r}_u \cdot \mathbf{r}_v}{||\mathbf{r}_u|| \cdot ||\mathbf{r}_v||} = \frac{\sum_{i \in I_{uv}} r_{ui} r_{vi}}{\sqrt{\sum_{i} r_{ui}^2} \sqrt{\sum_{i} r_{vi}^2}}$$

where $I_{uv}$ = items rated by both users u and v

**Pearson Correlation** (centered cosine):

$$\text{sim}(u, v) = \frac{\sum_{i \in I_{uv}} (r_{ui} - \bar{r}_u)(r_{vi} - \bar{r}_v)}{\sqrt{\sum_{i} (r_{ui} - \bar{r}_u)^2} \sqrt{\sum_{i} (r_{vi} - \bar{r}_v)^2}}$$

**Interpretation**: Pearson handles user bias (some users rate higher on average)

### 3. User-Based Collaborative Filtering

**Prediction** for user u on item i:

$$\hat{r}_{ui} = \bar{r}_u + \frac{\sum_{v \in N(u)} \text{sim}(u,v) \cdot (r_{vi} - \bar{r}_v)}{\sum_{v \in N(u)} |\text{sim}(u,v)|}$$

where $N(u)$ = k nearest neighbors of user u who rated item i

**Intuition**: Weighted average of neighbors' ratings, adjusted for their bias

### 4. Matrix Factorization (SVD)

**Decompose** R into user and item latent factors:

$$R \approx U \times V^T$$

where:
- $U \in \mathbb{R}^{n \times k}$: User factor matrix (n users, k latent features)
- $V \in \mathbb{R}^{m \times k}$: Item factor matrix (m items, k latent features)
- $k \ll \min(n, m)$: Latent dimensionality (e.g., 50-100)

**Prediction**:

$$\hat{r}_{ui} = \mu + b_u + b_i + \mathbf{u}_u^T \mathbf{v}_i$$

where:
- $\mu$: Global mean rating
- $b_u$: User bias (user u rates higher/lower than average)
- $b_i$: Item bias (item i is rated higher/lower than average)
- $\mathbf{u}_u^T \mathbf{v}_i$: Interaction between user and item latent factors

**Objective** (regularized squared error):

$$\min_{U,V,b} \sum_{(u,i) \in \text{known}} (r_{ui} - \hat{r}_{ui})^2 + \lambda(||U||^2 + ||V||^2 + ||b||^2)$$

**Optimization**: Stochastic Gradient Descent (SGD) or Alternating Least Squares (ALS)

### 5. Content-Based Filtering

**Item Profile**: Feature vector $\mathbf{x}_i \in \mathbb{R}^d$ (e.g., genre, keywords, test parameters)

**User Profile**: Weighted average of liked items:

$$\mathbf{p}_u = \frac{\sum_{i \in I_u} r_{ui} \cdot \mathbf{x}_i}{\sum_{i \in I_u} r_{ui}}$$

**Prediction**: Similarity between user profile and item:

$$\hat{r}_{ui} = \text{cosine}(\mathbf{p}_u, \mathbf{x}_i) = \frac{\mathbf{p}_u \cdot \mathbf{x}_i}{||\mathbf{p}_u|| \cdot ||\mathbf{x}_i||}$$

## 💻 Implementation from Scratch

### 📝 What's Happening in This Code?

**Purpose:** Build user-based and item-based collaborative filtering from scratch

**Key Points:**
- **UserBasedCF**: Find similar users via Pearson correlation, predict with weighted average
- **ItemBasedCF**: Precompute item similarity matrix, predict with weighted sum
- **Pearson**: Centers ratings (handles user bias)
- **k neighbors**: Limit to top-k most similar (reduces noise)
- **Fallback**: If no neighbors, use user/item mean

**Why This Matters:** 
- Understanding similarity metrics clarifies when each approach works
- Item-based often better for sparse data (items change less than users)
- Implementation shows computational bottlenecks (similarity computation)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.spatial.distance import cosine
from scipy.stats import pearsonr

sns.set_style('whitegrid')
np.random.seed(42)

class UserBasedCF:
    """User-based collaborative filtering."""
    
    def __init__(self, k=10, min_overlap=2):
        self.k = k  # Number of neighbors
        self.min_overlap = min_overlap  # Minimum co-rated items
        
    def fit(self, R):
        """R: user-item matrix (n_users × n_items), NaN for missing."""
        self.R = R
        self.user_means = np.nanmean(R, axis=1)
        return self
    
    def _pearson_similarity(self, u, v):
        """Pearson correlation between users u and v."""
        # Find co-rated items
        mask = ~np.isnan(self.R[u]) & ~np.isnan(self.R[v])
        if np.sum(mask) < self.min_overlap:
            return 0.0
        
        r_u = self.R[u, mask] - self.user_means[u]
        r_v = self.R[v, mask] - self.user_means[v]
        
        if np.std(r_u) == 0 or np.std(r_v) == 0:
            return 0.0
        
        return np.corrcoef(r_u, r_v)[0, 1]
    
    def predict(self, u, i):
        """Predict rating for user u on item i."""
        # Find users who rated item i
        rated_users = np.where(~np.isnan(self.R[:, i]))[0]
        if len(rated_users) == 0:
            return self.user_means[u]
        
        # Compute similarities
        sims = [(v, self._pearson_similarity(u, v)) for v in rated_users if v != u]
        sims = [(v, s) for v, s in sims if s > 0]  # Keep positive correlations
        
        if len(sims) == 0:
            return self.user_means[u]
        
        # Top-k neighbors
        sims = sorted(sims, key=lambda x: x[1], reverse=True)[:self.k]
        
        # Weighted average
        numerator = sum(s * (self.R[v, i] - self.user_means[v]) for v, s in sims)
        denominator = sum(abs(s) for _, s in sims)
        
        if denominator == 0:
            return self.user_means[u]
        
        return self.user_means[u] + numerator / denominator


class ItemBasedCF:
    """Item-based collaborative filtering."""
    
    def __init__(self, k=10, min_overlap=2):
        self.k = k
        self.min_overlap = min_overlap
        
    def fit(self, R):
        """Precompute item similarity matrix."""
        self.R = R
        self.item_means = np.nanmean(R, axis=0)
        self.n_items = R.shape[1]
        
        # Compute item similarity matrix
        self.sim_matrix = np.zeros((self.n_items, self.n_items))
        for i in range(self.n_items):
            for j in range(i+1, self.n_items):
                sim = self._cosine_similarity(i, j)
                self.sim_matrix[i, j] = sim
                self.sim_matrix[j, i] = sim
        
        return self
    
    def _cosine_similarity(self, i, j):
        """Cosine similarity between items i and j."""
        # Find users who rated both
        mask = ~np.isnan(self.R[:, i]) & ~np.isnan(self.R[:, j])
        if np.sum(mask) < self.min_overlap:
            return 0.0
        
        r_i = self.R[mask, i]
        r_j = self.R[mask, j]
        
        if np.linalg.norm(r_i) == 0 or np.linalg.norm(r_j) == 0:
            return 0.0
        
        return np.dot(r_i, r_j) / (np.linalg.norm(r_i) * np.linalg.norm(r_j))
    
    def predict(self, u, i):
        """Predict rating for user u on item i."""
        # Find items rated by user u
        rated_items = np.where(~np.isnan(self.R[u, :]))[0]
        if len(rated_items) == 0:
            return self.item_means[i]
        
        # Get similarities
        sims = [(j, self.sim_matrix[i, j]) for j in rated_items if j != i]
        sims = [(j, s) for j, s in sims if s > 0]
        
        if len(sims) == 0:
            return self.item_means[i]
        
        # Top-k similar items
        sims = sorted(sims, key=lambda x: x[1], reverse=True)[:self.k]
        
        # Weighted sum
        numerator = sum(s * self.R[u, j] for j, s in sims)
        denominator = sum(s for _, s in sims)
        
        if denominator == 0:
            return self.item_means[i]
        
        return numerator / denominator


print("✅ Collaborative filtering implementations complete!")
print("   - User-based: Find similar users (Pearson correlation)")
print("   - Item-based: Precompute item similarities (Cosine)")
print("   - Both use k-nearest neighbors for prediction")

## 🧪 Test on MovieLens-style Data

### 📝 What's Happening in This Code?

**Purpose:** Validate implementations on synthetic rating data

**Key Points:**
- **Synthetic ratings**: 100 users, 50 items, ~80% sparsity
- **Latent factors**: Users/items with 3 hidden preferences (genre-like)
- **Train/test split**: Hold out 20% for evaluation
- **MAE/RMSE**: Standard recommendation metrics
- **Comparison**: User-based vs item-based performance

**Why This Matters:** 
- Synthetic data with known structure validates algorithm
- Shows which approach works better for this sparsity level
- Establishes baseline before production library

In [ ]:
# Generate synthetic rating data with latent factors
n_users = 100
n_items = 50
n_factors = 3
sparsity = 0.8  # 80% missing

# Latent factor matrices
U_true = np.random.randn(n_users, n_factors)
V_true = np.random.randn(n_items, n_factors)

# Generate ratings R = U × V^T + noise
R_true = U_true @ V_true.T + np.random.randn(n_users, n_items) * 0.5
R_true = np.clip(R_true, 1, 5)  # Ratings 1-5

# Create sparse matrix
R = R_true.copy()
mask = np.random.rand(n_users, n_items) < sparsity
R[mask] = np.nan

print(f"📊 Synthetic Rating Matrix:")
print(f"   Shape: {R.shape}")
print(f"   Sparsity: {np.isnan(R).mean()*100:.1f}%")
print(f"   Rating range: [{np.nanmin(R):.2f}, {np.nanmax(R):.2f}]")

# Split train/test
observed = ~np.isnan(R)
indices = np.argwhere(observed)
np.random.shuffle(indices)
n_test = int(0.2 * len(indices))
test_indices = indices[:n_test]

R_train = R.copy()
R_test_values = []
for u, i in test_indices:
    R_test_values.append((u, i, R_train[u, i]))
    R_train[u, i] = np.nan

print(f"\n   Train ratings: {(~np.isnan(R_train)).sum()}")
print(f"   Test ratings: {len(R_test_values)}")

# Train models
print("\n🔧 Training models...")
user_cf = UserBasedCF(k=15).fit(R_train)
item_cf = ItemBasedCF(k=15).fit(R_train)

# Evaluate
def evaluate(model, test_data):
    predictions = []
    actuals = []
    for u, i, r_true in test_data:
        r_pred = model.predict(u, i)
        predictions.append(r_pred)
        actuals.append(r_true)
    
    predictions = np.array(predictions)
    actuals = np.array(actuals)
    
    mae = np.mean(np.abs(predictions - actuals))
    rmse = np.sqrt(np.mean((predictions - actuals)**2))
    
    return mae, rmse, predictions, actuals

mae_user, rmse_user, preds_user, actuals_user = evaluate(user_cf, R_test_values)
mae_item, rmse_item, preds_item, actuals_item = evaluate(item_cf, R_test_values)

print(f"\n📊 Evaluation Results:")
print(f"\n   User-Based CF:")
print(f"      MAE:  {mae_user:.3f}")
print(f"      RMSE: {rmse_user:.3f}")
print(f"\n   Item-Based CF:")
print(f"      MAE:  {mae_item:.3f}")
print(f"      RMSE: {rmse_item:.3f}")

# Visualize
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Prediction vs Actual (User-based)
ax = axes[0]
ax.scatter(actuals_user, preds_user, alpha=0.3, s=10)
ax.plot([1, 5], [1, 5], 'r--', linewidth=2, label='Perfect prediction')
ax.set_xlabel('Actual Rating')
ax.set_ylabel('Predicted Rating')
ax.set_title(f'User-Based CF\nRMSE={rmse_user:.3f}')
ax.legend()
ax.grid(True, alpha=0.3)

# Prediction vs Actual (Item-based)
ax = axes[1]
ax.scatter(actuals_item, preds_item, alpha=0.3, s=10, color='orange')
ax.plot([1, 5], [1, 5], 'r--', linewidth=2, label='Perfect prediction')
ax.set_xlabel('Actual Rating')
ax.set_ylabel('Predicted Rating')
ax.set_title(f'Item-Based CF\nRMSE={rmse_item:.3f}')
ax.legend()
ax.grid(True, alpha=0.3)

# Error distribution
ax = axes[2]
errors_user = preds_user - actuals_user
errors_item = preds_item - actuals_item
ax.hist(errors_user, bins=30, alpha=0.5, label='User-based', color='blue')
ax.hist(errors_item, bins=30, alpha=0.5, label='Item-based', color='orange')
ax.axvline(0, color='red', linestyle='--', linewidth=2, label='Zero error')
ax.set_xlabel('Prediction Error')
ax.set_ylabel('Frequency')
ax.set_title('Error Distribution')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 🏭 Post-Silicon Application: Test Failure Prediction

### 📝 What's Happening in This Code?

**Purpose:** Predict which tests will fail for new devices (skip likely passing tests)

**Key Points:**
- **Matrix R**: Devices × Tests (1=fail, 0=pass, NaN=not run)
- **Collaborative insight**: Devices with similar past failures likely fail same tests
- **Business value**: Test time reduction by skipping likely-passing tests
- **Risk**: Must not skip tests that will fail (false negatives costly)
- **Threshold tuning**: Predict fail if score > 0.5 (adjustable for precision/recall)

**Why This Matters:** 
- Reduces ATE time (expensive resource)
- Enables adaptive test flows (device-specific)
- Maintains quality (catch all failures)

In [ ]:
# Generate semiconductor test failure data
n_devices = 500
n_tests = 30
sparsity = 0.7  # 70% tests not run yet

# Ground truth failure patterns (3 failure modes)
failure_mode_1 = [1, 2, 5, 8, 12]  # Process defect
failure_mode_2 = [3, 7, 11, 15, 20]  # Frequency issue
failure_mode_3 = [4, 9, 13, 18, 25]  # Thermal issue

R_psv = np.zeros((n_devices, n_tests))
for device in range(n_devices):
    # Assign to failure mode(s) or none
    if np.random.rand() < 0.10:
        R_psv[device, failure_mode_1] = 1
    if np.random.rand() < 0.08:
        R_psv[device, failure_mode_2] = 1
    if np.random.rand() < 0.05:
        R_psv[device, failure_mode_3] = 1
    
    # Random failures (noise)
    n_random = np.random.poisson(0.5)
    random_tests = np.random.choice(n_tests, min(n_random, 5), replace=False)
    R_psv[device, random_tests] = 1

# Create sparse matrix (only some tests run)
R_psv_observed = R_psv.copy()
mask = np.random.rand(n_devices, n_tests) < sparsity
R_psv_observed[mask] = np.nan

print(f"🔬 Test Failure Matrix:")
print(f"   Devices: {n_devices}, Tests: {n_tests}")
print(f"   Sparsity: {np.isnan(R_psv_observed).mean()*100:.1f}% (tests not run)")
print(f"   Overall failure rate: {R_psv.mean()*100:.1f}%")

# Split: first 400 devices for training, last 100 for testing
R_train_psv = R_psv_observed[:400, :].copy()
R_test_psv = R_psv_observed[400:, :].copy()
R_test_ground_truth = R_psv[400:, :]

print(f"\n   Training devices: 400")
print(f"   Test devices: 100")

# Train item-based model (items=tests)
print("\n🔧 Training test failure predictor...")
model_psv = ItemBasedCF(k=10, min_overlap=5).fit(R_train_psv)

# Predict for test devices
predictions_psv = []
actuals_psv = []
for device_id in range(100):
    for test_id in range(n_tests):
        if ~np.isnan(R_test_psv[device_id, test_id]):
            continue  # Already ran
        
        pred = model_psv.predict(device_id + 400, test_id)  # Adjust index
        actual = R_test_ground_truth[device_id, test_id]
        
        predictions_psv.append(pred)
        actuals_psv.append(actual)

predictions_psv = np.array(predictions_psv)
actuals_psv = np.array(actuals_psv)

# Binary classification metrics (threshold=0.5)
pred_binary = (predictions_psv > 0.5).astype(int)
accuracy = np.mean(pred_binary == actuals_psv)
precision = np.sum((pred_binary == 1) & (actuals_psv == 1)) / max(np.sum(pred_binary == 1), 1)
recall = np.sum((pred_binary == 1) & (actuals_psv == 1)) / max(np.sum(actuals_psv == 1), 1)
f1 = 2 * precision * recall / max(precision + recall, 1e-10)

print(f"\n📊 Test Failure Prediction:")
print(f"   Accuracy:  {accuracy*100:.1f}%")
print(f"   Precision: {precision*100:.1f}% (predicted fails that are actual fails)")
print(f"   Recall:    {recall*100:.1f}% (actual fails that were predicted)")
print(f"   F1 Score:  {f1:.3f}")

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# ROC-like curve (vary threshold)
ax = axes[0]
thresholds = np.linspace(0, 1, 50)
precisions = []
recalls = []
for thresh in thresholds:
    pred_bin = (predictions_psv > thresh).astype(int)
    tp = np.sum((pred_bin == 1) & (actuals_psv == 1))
    fp = np.sum((pred_bin == 1) & (actuals_psv == 0))
    fn = np.sum((pred_bin == 0) & (actuals_psv == 1))
    
    prec = tp / max(tp + fp, 1)
    rec = tp / max(tp + fn, 1)
    precisions.append(prec)
    recalls.append(rec)

ax.plot(recalls, precisions, linewidth=2, marker='o', markersize=3)
ax.set_xlabel('Recall (Catch Failures)')
ax.set_ylabel('Precision (Avoid False Alarms)')
ax.set_title('Precision-Recall Curve\n(Test Failure Prediction)')
ax.grid(True, alpha=0.3)

# Confusion matrix
ax = axes[1]
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(actuals_psv, pred_binary)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax,
           xticklabels=['Pass', 'Fail'], yticklabels=['Pass', 'Fail'])
ax.set_xlabel('Predicted')
ax.set_ylabel('Actual')
ax.set_title(f'Confusion Matrix\n(Threshold=0.5)')

plt.tight_layout()
plt.show()

print("\n💡 Business Insights:")
print(f"   - High recall ({recall*100:.1f}%) means we catch most failures")
print(f"   - Precision {precision*100:.1f}% indicates false positive rate")
print(f"   - Can skip predicted-passing tests to reduce ATE time")
print(f"   - Tune threshold based on cost of missed failures vs unnecessary tests")

## 🔧 Matrix Factorization with Surprise Library

### 📝 What's Happening in This Code?

**Purpose:** Use production-ready SVD implementation from Surprise library

**Key Points:**
- **SVD**: Decomposes rating matrix into user/item latent factors
- **Biases**: Models global, user, and item biases
- **SGD optimization**: Efficient learning with regularization
- **Cross-validation**: 5-fold CV for robust evaluation
- **Comparison**: SVD vs KNN (collaborative filtering)

**Why This Matters:** 
- Production systems use matrix factorization (scales to millions)
- SVD captures latent preferences (user taste, item genres)
- Surprise library handles sparse data efficiently

In [ ]:
from surprise import Dataset, Reader, SVD, KNNBasic
from surprise.model_selection import cross_validate, train_test_split
from surprise import accuracy

# Convert to Surprise format
ratings_list = []
for u in range(n_users):
    for i in range(n_items):
        if ~np.isnan(R[u, i]):
            ratings_list.append((u, i, R[u, i]))

reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(pd.DataFrame(ratings_list, columns=['userID', 'itemID', 'rating']), reader)

print(f"📊 Loaded {len(ratings_list)} ratings into Surprise")

# Train SVD model
print("\n🔧 Training SVD (Matrix Factorization)...")
svd = SVD(n_factors=10, n_epochs=20, lr_all=0.005, reg_all=0.02, random_state=42)
cv_results_svd = cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=5, verbose=False)

print(f"\n   SVD Results (5-fold CV):")
print(f"      RMSE: {cv_results_svd['test_rmse'].mean():.3f} ± {cv_results_svd['test_rmse'].std():.3f}")
print(f"      MAE:  {cv_results_svd['test_mae'].mean():.3f} ± {cv_results_svd['test_mae'].std():.3f}")

# Compare with KNN
print("\n🔧 Training KNN (User-Based CF)...")
knn = KNNBasic(k=15, sim_options={'name': 'pearson', 'user_based': True})
cv_results_knn = cross_validate(knn, data, measures=['RMSE', 'MAE'], cv=5, verbose=False)

print(f"\n   KNN Results (5-fold CV):")
print(f"      RMSE: {cv_results_knn['test_rmse'].mean():.3f} ± {cv_results_knn['test_rmse'].std():.3f}")
print(f"      MAE:  {cv_results_knn['test_mae'].mean():.3f} ± {cv_results_knn['test_mae'].std():.3f}")

# Visualize latent factors (train full SVD)
trainset = data.build_full_trainset()
svd.fit(trainset)

# Extract user and item factors
user_factors = svd.pu  # n_users × n_factors
item_factors = svd.qi  # n_items × n_factors

print(f"\n📊 Learned Latent Factors:")
print(f"   User factors: {user_factors.shape}")
print(f"   Item factors: {item_factors.shape}")

# Visualize first 2 latent dimensions
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# User embeddings
ax = axes[0]
ax.scatter(user_factors[:, 0], user_factors[:, 1], alpha=0.5, s=30)
ax.set_xlabel('Latent Factor 1')
ax.set_ylabel('Latent Factor 2')
ax.set_title('User Embeddings (First 2 Factors)')
ax.grid(True, alpha=0.3)

# Item embeddings
ax = axes[1]
ax.scatter(item_factors[:, 0], item_factors[:, 1], alpha=0.5, s=30, color='orange')
ax.set_xlabel('Latent Factor 1')
ax.set_ylabel('Latent Factor 2')
ax.set_title('Item Embeddings (First 2 Factors)')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n✅ Surprise Library Results:")
print("   - SVD: Matrix factorization with latent factors")
print("   - KNN: Neighborhood-based collaborative filtering")
print("   - Production-ready implementations with cross-validation")

## 🎯 Real-World Project Ideas

### Post-Silicon Validation Projects

1. **Adaptive Test Flow Recommender** 💰 $15M+ Test Efficiency
   - **Objective**: Predict per-device test sequence, skip likely-passing tests
   - **Features**: Device parametric signature + historical test results
   - **Success Metric**: 30% test time reduction, 99.9% defect capture
   - **Implementation**: Item-based CF (tests=items), predict failure prob, adaptive thresholds

2. **Failure Analysis Strategy Recommender** 💰 $10M+ Debug Acceleration
   - **Objective**: Suggest FA techniques based on similar failure signatures
   - **Features**: Parametric test results + wafer spatial + lot context
   - **Success Metric**: 40% FA time reduction, 95% root cause match
   - **Implementation**: Hybrid (content-based on params + collaborative on past FA success)

3. **Equipment Health Predictor** 💰 $20M+ Downtime Prevention
   - **Objective**: Recommend maintenance based on tool parameter drift
   - **Features**: 100+ tool sensors + recipe parameters + chamber usage
   - **Success Metric**: 7-day advance warning, <5% false alarms
   - **Implementation**: SVD on tool×parameter matrix, detect abnormal latent factors

4. **Cross-Product Yield Transfer** 💰 $50M+ Ramp Acceleration
   - **Objective**: Recommend initial test limits for new product based on similar products
   - **Features**: Product specs + package type + process node + test results
   - **Success Metric**: 50% faster yield learning, 90% limit accuracy
   - **Implementation**: Content-based (product similarity) + transfer learning

### General AI/ML Projects

5. **E-Commerce Product Recommender** 💰 $100M+ Revenue
   - **Objective**: Personalized product recommendations
   - **Features**: Purchase history + browsing + demographics + product attributes
   - **Success Metric**: 25% conversion rate increase, 20% AOV lift
   - **Implementation**: Hybrid (CF for personalization + content for cold-start)

6. **Video Streaming Recommender** 💰 $200M+ Engagement
   - **Objective**: Recommend movies/shows to increase watch time
   - **Features**: Viewing history + ratings + genres + actors + descriptions
   - **Success Metric**: 30% more watch time, 50% click-through rate
   - **Implementation**: Deep learning hybrid (embeddings + sequence models)

7. **Job-Candidate Matching** 💰 $50M+ Placement Success
   - **Objective**: Recommend candidates to jobs and vice versa
   - **Features**: Skills + experience + education + company culture + job descriptions
   - **Success Metric**: 40% placement rate increase, 80% 1-year retention
   - **Implementation**: Two-way recommendations (candidates+jobs as users+items)

8. **News Article Recommender** 💰 $80M+ Ad Revenue
   - **Objective**: Personalized news feed with diversity and timeliness
   - **Features**: Reading history + click behavior + article topics + recency
   - **Success Metric**: 50% more engagement, 30% session time increase
   - **Implementation**: Contextual bandits + explore-exploit for fresh content

## 🔍 Key Takeaways

### ✅ When to Use Recommender Systems
- **Sparse interactions**: Users rate/interact with <1% of items
- **Implicit feedback**: Clicks, views, purchases (not just explicit ratings)
- **Personalization**: Each user has unique preferences
- **Large item catalogs**: Millions of products/videos/articles
- **Cold-start problem**: New users/items with no history (use content-based)

### ❌ Limitations
- **Cold-start**: New users/items hard to recommend
- **Popularity bias**: Tends to recommend popular items
- **Filter bubble**: Users only see similar items (lack diversity)
- **Data sparsity**: Need sufficient interactions (>5 per user)
- **No explanations**: Matrix factorization latent factors not interpretable

### 🔧 Best Practices
1. **Start with item-based CF**: Works well for sparse data, more stable than user-based
2. **Use matrix factorization for scale**: SVD/ALS handles millions of users
3. **Hybrid approaches**: Combine CF + content-based for cold-start
4. **Implicit feedback**: Use clicks/views (not just ratings) for more data
5. **Diversity**: Add exploration (not just exploitation) to avoid filter bubbles
6. **A/B testing**: Always test recommendations with real users

### 📊 Approach Comparison

| Approach | Pros | Cons | Use Case |
|----------|------|------|----------|
| **User-Based CF** | Simple, explainable | Scalability, user sparsity | Small datasets |
| **Item-Based CF** | Stable, scalable | Item cold-start | Large catalogs |
| **Matrix Factorization** | Handles sparsity, scalable | Black box, cold-start | Millions of users |
| **Content-Based** | No cold-start, explainable | Overspecialization | Rich item metadata |
| **Hybrid** | Best of both worlds | Complex, tuning needed | Production systems |

### 🚀 Next Steps
- **Deep learning**: Neural collaborative filtering, autoencoders
- **Contextual bandits**: Exploration-exploitation for fresh content
- **Graph neural networks**: User-item-context graphs
- **Sequential models**: RNNs for session-based recommendations

### 🔬 Production Considerations
- **Online learning**: Update models incrementally with new interactions
- **Latency**: Precompute similarities, cache predictions
- **Diversity**: Add randomness, novelty, serendipity
- **Bias mitigation**: Fair recommendations across demographics
- **Evaluation**: Use online metrics (CTR, conversion) not just RMSE